In [1]:
import torch
from pyqcu.ascend import dslash
from pyqcu.ascend import inverse
dof = 12
# latt_size = (16, 16, 16, 16)
# latt_size = (4, 8, 8, 8)
latt_size = (8, 8, 8, 4)
# latt_size = (8, 8, 8, 8)
# latt_size = (4, 4, 4, 4)
kappa = 0.125
# dtype = torch.complex128
dtype = torch.complex64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
# Initialize lattice gauge theory
wilson = dslash.wilson_parity(
    latt_size=latt_size,
    kappa=kappa,
    dtype=dtype,
    device=device,
    verbose=False
)
clover = dslash.clover_parity(
    latt_size=latt_size,
    kappa=kappa,
    dtype=dtype,
    device=device,
    verbose=False
)
U = wilson.generate_gauge_field(sigma=0.1, seed=42)
null_vecs = torch.randn(dof, 4, 3, latt_size[3], latt_size[2], latt_size[1], latt_size[0],
                        dtype=dtype, device=device)
clover_term = clover.make_clover(U=U)
clover_term = clover.add_I(clover=clover_term)  # needed!


b = torch.randn_like(null_vecs[0])
U_eo = dslash.xxxtzyx2pxxxtzyx(input_array=U)
clover_eo = dslash.xxxtzyx2pxxxtzyx(input_array=clover_term)

@xxxtzyx2pxxxtzyx......
Splited Array Shape: torch.Size([2, 3, 3, 4, 4, 8, 8, 4])
@xxxtzyx2pxxxtzyx......
Splited Array Shape: torch.Size([2, 4, 3, 4, 3, 4, 8, 8, 4])


In [2]:
from pyqcu.ascend import inverse
mg = inverse.mg(b=b, wilson=wilson, U_eo=U_eo,
                clover=clover, clover_eo=clover_eo)


Building grid hierarchy:
  Level 0: 8x8x8x4
  Level 1: 4x4x4x2
  Level 2: 2x2x2x2
[[4, 8, 8, 8], [2, 4, 4, 4], [2, 2, 2, 2]]
@xxxtzyx2pxxxtzyx......
Splited Array Shape: torch.Size([2, 4, 3, 4, 8, 8, 4])
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@pxxxtzyx2xxxtzyx......
Restored Array Shape: torch.Size([4, 3, 4, 8, 8, 8])
@xxxtzyx2pxxxtzyx......
Splited Array Shape: torch.Size([2, 4, 3, 4, 8, 8, 4])
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@pxxxtzyx2xxxtzyx......
Restored Array Shape: torch.Size([4, 3, 4, 8, 8, 8])
@xxxtzyx2pxxxtzyx......
Splited Array Shape: torch.Size([2, 4, 3, 4, 8, 8, 4])
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@pxxxtzyx2xxxtzyx......
Restored Array Shape: torch.Size([4, 3, 4, 8, 8, 8])
@xxxtzyx2pxxxtzyx......
Splited Array Shape: torch.Size([2, 4, 3, 4, 8, 8, 4])
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@give_eo_mask......
@pxxxtzy